In [9]:
import numpy as np
import awkward as ak
import uproot, icp, json


np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

In [2]:
availableOverlapIDs = []
with open("comparisonData/sensors-1.00/rawData/detectorOverlapsIdeal.json") as f:
    idealOverlaps = json.load(f)
    for overlap in idealOverlaps:
        availableOverlapIDs.append(idealOverlaps[overlap]["overlapID"])


In [ ]:
#! Attention! Are they sorted? I don't think so!

# read from root and sort to numpy, process one root file ENTTIRELY at a time an then write all np arrays to disk
# only then read next file (saves on IO)
filename = "comparisonData/sensors-1.00/rawData/Lumi_Pairs_*.root"

runIndex = 0
maxNoOfFiles = 0
npyOutputDir = "comparisonData/sensors-1.00/npPairs"

for arrays in uproot.iterate(
    filename,
    [
        "PndLmdHitPair._overlapID",
        "PndLmdHitPair._hit1",
        "PndLmdHitPair._hit2",
    ],
    # library="np", # DONT use numpy yet, we need the awkward array for the TVector3
    allow_missing=True,  # some files may be empty, skip those):
):
    runIndex += 1

    # apply a mask for at least one overlap (since they come event-based, there
    # may actually be not hit in a given event)

    overlapIDs = np.array(ak.flatten(arrays["PndLmdHitPair._overlapID"]))
    hit1x = ak.flatten(arrays["PndLmdHitPair._hit1"].fX)
    hit1y = ak.flatten(arrays["PndLmdHitPair._hit1"].fY)
    hit1z = ak.flatten(arrays["PndLmdHitPair._hit1"].fZ)
    hit2x = ak.flatten(arrays["PndLmdHitPair._hit2"].fX)
    hit2y = ak.flatten(arrays["PndLmdHitPair._hit2"].fY)
    hit2z = ak.flatten(arrays["PndLmdHitPair._hit2"].fZ)

    hit1 = np.array((hit1x, hit1y, hit1z)).T
    hit2 = np.array((hit2x, hit2y, hit2z)).T

    distVec = np.linalg.norm(hit1 - hit2, axis=1)

    arr = np.array((overlapIDs, hit1x, hit1y, hit1z, hit2x, hit2y, hit2z, distVec)).T

    for overlap in availableOverlapIDs:
        mask = arr[:, 0] == overlap
        thisOverlapsArray = arr[mask][:, 1:]

        # read array from disk
        fileName = f"{npyOutputDir}/pairs-{overlap}.npy"

        try:
            oldContent = np.load(fileName)
        # first run, file not already present
        except:
            oldContent = np.empty((0, 7))

        # merge
        newContent = np.concatenate((oldContent, thisOverlapsArray))

        # write back to disk
        np.save(file=fileName, arr=newContent, allow_pickle=False)

    if runIndex == maxNoOfFiles:
        break


In [ ]:
# Test, did that work?
arr1 = np.load('comparisonData/sensors-1.00/rawData/npPairs/pairs-0.npy').T
arr2 = np.load('comparisonData/sensors-1.00/npPairs/pairs-0.npy')
print(arr1[0])
print(arr2[0])

# well, not identical, but ... similar? test aligener and see

In [22]:
def dynamicCut(hitPairs, cutPercent=2):

    if cutPercent == 0:
        return hitPairs

    # calculate center of mass of differences
    dRaw = hitPairs[:, 3:6] - hitPairs[:, :3]
    com = np.average(dRaw, axis=0)

    # shift newhit2 by com of differences
    newhit2 = hitPairs[:, 3:6] - com

    # calculate new distance for cut
    dRaw = newhit2 - hitPairs[:, :3]
    newDist = np.power(dRaw[:, 0], 2) + np.power(dRaw[:, 1], 2)

    # sort by distance and cut some percent from end (discard outliers)
    cut = int(len(hitPairs) * cutPercent / 100.0)
    # sort by new distance
    hitPairs = hitPairs[newDist.argsort()]
    # cut off largest distances, NOT lowest
    hitPairs = hitPairs[:-cut]

    return hitPairs


In [4]:
# read required external data
def loadMatrices(filename):
    with open(filename) as f:
        result = json.load(f)
    for key, value in result.items():
        result[key] = np.array(value).reshape(4, 4)
    return result

In [23]:
PairData = np.load('comparisonData/sensors-1.00/npPairs/pairs-2.npy')
thisOverlap = 2
idealDetectorMatrices = loadMatrices("comparisonData/oldInput/detectorMatricesIdeal.json")
use2D = True

with open("comparisonData/modules-1.00/rawData/detectorOverlapsIdeal.json") as f:
    idealOverlapInfos = json.load(f)

# apply dynamic cut
PairData = dynamicCut(PairData , 2)

def findMatrix():

    # if idealOverlapInfos is None or PairData is None:
    #     raise Exception(f"Error! Please load ideal detector matrices and numpy pairs!")

    # if len(idealDetectorMatrices) < 1:
    #     raise Exception("ERROR! Please set ideal detector matrices!")

    # Make C a homogeneous representation of hits1 and hits2
    hit1H = np.ones((len(PairData), 4))
    hit1H[:, 0:3] = PairData[:, :3]

    hit2H = np.ones((len(PairData), 4))
    hit2H[:, 0:3] = PairData[:, 3:6]

    # Attention! Always transform to module-local system,
    # otherwise numerical errors will make the ICP matrices unusable!
    # (because z is at 11m, while x is 30cm and y is 0)
    # also, we're ignoring z distance, which we can not do if we're in
    # PND global, due to the 40mrad rotation.
    transformToLocalSensor = True
    if transformToLocalSensor:
        icpDimension = 2
        # get matrix lmd to module
        modulePath = idealOverlapInfos[str(thisOverlap)]["pathModule"]
        matToModule = idealDetectorMatrices[modulePath]

        # invert to transform pairs from lmd to sensor
        toModInv = np.linalg.inv(matToModule)

        # Transform vectors (remember, C and D are vectors of vectors = matrices!)
        hit1T = np.matmul(toModInv, hit1H.T).T
        hit2T = np.matmul(toModInv, hit2H.T).T

    else:
        print("WARNING! ICP working in Panda global, NOT sensor local.")
        print("This will likely produce wrong overlap matrices!")
        hit1T = hit1H
        hit2T = hit2H

    if use2D:
        icpDimension = 2
        # make 2D versions for ICP
        A = hit1T[:, :2]
        B = hit2T[:, :2]
    else:
        icpDimension = 3
        # make 3D versions for ICP
        A = hit1T[:, :3]
        B = hit2T[:, :3]

    # find ideal transformation
    T, _, _ = icp.best_fit_transform(A, B)

    # copy 3x3 Matrix to 4x4 Matrix
    if icpDimension == 2:
        M = np.identity(4)
        M[:2, :2] = T[:2, :2]
        M[:2, 3] = T[:2, 2]
        thisOverlapMatrix = M

    elif icpDimension == 3:
        thisOverlapMatrix = T

    transformResultToPND = True
    if transformResultToPND:
        # remember, matToModule goes from Pnd->Module
        # base trafo is T A T^-1,
        # T = Pnd->Module
        thisOverlapMatrix = (
            (matToModule) @ thisOverlapMatrix @ np.linalg.inv(matToModule)
        )
    print(thisOverlapMatrix*1e4)

findMatrix()

[[ 9999.99979     -2.047935     0.000008  -120.580534]
 [    2.047935  9999.99979     -0.082003   358.068146]
 [    0.000008     0.082003 10000.           4.828246]
 [    0.           0.           0.       10000.      ]]


In [20]:
# then, use MT:
# each thread reads one npy file and processes, submit to thread pool
mOld = loadMatrices('comparisonData/sensors-1.00/alMat-sensorOverlaps-1.00.json')
print(mOld['2']*1e4)

# looking good so far

[[ 9999.999787    -2.064078     0.000009  -120.515935]
 [    2.064078  9999.999787    -0.082649   358.208057]
 [    0.000009     0.082649 10000.           4.825659]
 [    0.           0.           0.       10000.      ]]


In [ ]:
mMis = loadMatrices('comparisonData/misMatrices/misMat-sensors-1.00.json')
print(mMis['0']*1e4)
